### Análisis y Predicción de Ventas en una Tienda de Retail (Core)
Parte III: Transformación y Análisis Avanzado de Datos con Pandas

En esta tercera parte del proyecto, continuaremos trabajando con el dataset de ventas que utilizamos en la Parte II. En esta fase, aplicaremos técnicas avanzadas de transformación y análisis de datos utilizando las nuevas habilidades adquiridas en Pandas, tales como agrupaciones complejas y el uso del método apply. Nos enfocaremos en extraer insights más profundos y preparar los datos para futuros análisis y modelos predictivos.

### Transformación de Datos
Crea nuevas columnas: Basándonos en los datos existentes, crea nuevas columnas que sean útiles para el análisis. Por ejemplo, calcula el ingreso total por venta y normaliza las ventas.

In [11]:
import pandas as pd
df = pd.read_csv(r'C:\Users\Gonzalo\OneDrive\Documentos\repos\mitic-data-science-gonza\data\retail_sales_dataset.csv')
df.head()

,Transaction ID,Date,Customer ID,Gender,Age,Product Category,Quantity,Price per Unit,Total Amount
0,1,2023-11-24,CUST001,Male,34,Beauty,3,50,150
1,2,2023-02-27,CUST002,Female,26,Clothing,2,500,1000
2,3,2023-01-13,CUST003,Male,50,Electronics,1,30,30
3,4,2023-05-21,CUST004,Male,37,Clothing,1,500,500
4,5,2023-05-06,CUST005,Male,30,Beauty,2,50,100


: 

In [2]:
# Convertir la columna 'Date' en un tipo de dato datetime (si aún no lo es)
df['Date'] = pd.to_datetime(df['Date'])

# Definir una función para asignar la temporada en base al mes
def asignar_temporada(mes):
    if mes in [3, 4, 5]:
        return 'Spring'
    elif mes in [6, 7, 8]:
        return 'Summer'
    elif mes in [9, 10, 11]:
        return 'Fall'
    else:
        return 'Winter'

# Aplicar la función para crear la nueva columna 'Sales Season'
df['Sales Season'] = df['Date'].dt.month.apply(asignar_temporada)

# Mostrar los primeros registros para verificar
print(df[['Date', 'Sales Season']].head())


        Date Sales Season
0 2023-11-24         Fall
1 2023-02-27       Winter
2 2023-01-13       Winter
3 2023-05-21       Spring
4 2023-05-06       Spring


In [3]:
## Normalizar las ventas
max_value = df['Total Amount'].max()
min_value = df['Total Amount'].min()
df['norm_sales'] = df['Total Amount'].apply(lambda x: (x - min_value) / (max_value - min_value))


Clasifica los datos: Crea una columna que clasifique las ventas en categorías significativas (e.g., ‘Alta’, ‘Media’, ‘Baja’).

In [4]:
# Definir categorias basado en cuartiles
def categorize(value, q1, q3):
    if value <= q1:
        return 'Low'
    elif value <= q3:
        return 'Moderate'
    else:
        return 'High'
q1 = df['norm_sales'].quantile(0.25)
q3 = df['norm_sales'].quantile(0.75)

df['sales_category'] = df['norm_sales'].apply(categorize, args=(q1, q3))
df[['Total Amount', 'norm_sales', 'sales_category']]

,Total Amount,norm_sales,sales_category
0,150,0.063291,Moderate
1,1000,0.493671,High
2,30,0.002532,Low
3,500,0.240506,Moderate
4,100,0.037975,Moderate
...,...,...,...
995,50,0.012658,Low
996,90,0.032911,Moderate
997,100,0.037975,Moderate
998,150,0.063291,Moderate


### Agrupación y Agregación

Agrupación por múltiples columnas: Realiza agrupaciones por categorías como producto y tienda, producto y mes, etc

In [5]:
df_producto_genero = df.groupby(['Product Category', 'Gender']).value_counts()
df_product_season = df.groupby(['Product Category', 'Sales Season']).value_counts()
df_producto_genero, df_product_season

(Product Category  Gender  Transaction ID  Date        Customer ID  Age  Quantity  Price per Unit  Total Amount  Sales Season  norm_sales  sales_category
 Beauty            Female  6               2023-04-25  CUST006      45   1         30              30            Spring        0.002532    Low               1
                           21              2023-01-14  CUST021      50   1         500             500           Winter        0.240506    Moderate          1
                           25              2023-12-26  CUST025      64   1         50              50            Winter        0.012658    Low               1
                           27              2023-08-03  CUST027      38   2         25              50            Summer        0.012658    Low               1
                           28              2023-04-23  CUST028      43   1         500             500           Spring        0.240506    Moderate          1
                                                   

Aplicar funciones de agregación: Utiliza funciones como sum, mean, count, min, max, std, y var para obtener estadísticas descriptivas de cada grupo.

In [6]:
# Agrupar por la categoría de producto y aplicar funciones de agregación
estadisticas_ventas = df.groupby('Product Category')['Total Amount'].agg(
    Total_Suma='sum',
    Promedio='mean',
    Conteo='count',
    Minimo='min',
    Maximo='max',
    Desviacion='std',
    Varianza='var'
)

print(estadisticas_ventas)


                  Total_Suma    Promedio  Conteo  Minimo  Maximo  Desviacion  \
Product Category                                                               
Beauty                143515  467.475570     307      25    2000  563.612788   
Clothing              155580  443.247863     351      25    2000  550.695917   
Electronics           156905  458.786550     342      25    2000  567.540150   

                       Varianza  
Product Category                 
Beauty            317659.374401  
Clothing          303265.992674  
Electronics       322101.822341  


### Análisis Personalizado con apply
Función personalizada: Aplica funciones personalizadas para realizar análisis específicos que no se pueden lograr con las funciones de agregación estándar.

In [8]:
# Definir una función personalizada para calcular el Coeficiente de Variación
def coeficiente_variacion(series):
    return series.std() / series.mean()

# Aplicar la función personalizada usando el método agg
analisis_personalizado = df.groupby('Product Category')['Total Amount'].agg(
    Total_Suma='sum',
    Promedio='mean',
    Coef_Variacion=coeficiente_variacion
)

print(analisis_personalizado)


                  Total_Suma    Promedio  Coef_Variacion
Product Category                                        
Beauty                143515  467.475570        1.205652
Clothing              155580  443.247863        1.242411
Electronics           156905  458.786550        1.237046


Ejemplo de uso avanzado: Calcula la desviación de cada venta respecto a la media de su grupo.

In [10]:
# Agrupar por categoría de producto y calcular la media de cada grupo
media_por_grupo = df.groupby('Product Category')['Total Amount'].transform('mean')

# Calcular la desviación de cada venta respecto a la media de su grupo
df['Desviacion_Venta'] = df['Total Amount'] - media_por_grupo

print(df[['Product Category', 'Total Amount', 'Desviacion_Venta']].head())



  Product Category  Total Amount  Desviacion_Venta
0           Beauty           150       -317.475570
1         Clothing          1000        556.752137
2      Electronics            30       -428.786550
3         Clothing           500         56.752137
4           Beauty           100       -367.475570


In [ ]:
df.to